# Модуль 4.1 — RAG: загрузка и чанкинг

**Цель:** научиться загружать документы и нарезать их на чанки для последующего поиска.

**Что сделаем:**
- установим библиотеки
- загрузим документы из папки
- разделим текст на чанки
- разберем параметры чанкинга

In [ ]:
%pip -q install -U \
  langchain \
  langchain-community \
  unstructured \
  pydantic==2.12.3

## Если Colab ругается на зависимости

Иногда в Colab установлены библиотеки с жёсткими версиями (например, `google-adk` и `opentelemetry`).
Если после установки появляются конфликты — зафиксируйте совместимые версии ниже.

In [ ]:
%pip -q install -U \
  opentelemetry-api==1.37.0 \
  opentelemetry-sdk==1.37.0 \
  opentelemetry-proto==1.37.0 \
  opentelemetry-exporter-otlp-proto-common==1.37.0 \
  opentelemetry-exporter-otlp-proto-grpc==1.37.0

# На случай, если в окружении уже стоит более новая версия
def _force_pins():
    import sys, subprocess
    pkgs = [
        "opentelemetry-api==1.37.0",
        "opentelemetry-sdk==1.37.0",
        "opentelemetry-proto==1.37.0",
        "opentelemetry-exporter-otlp-proto-common==1.37.0",
        "opentelemetry-exporter-otlp-proto-grpc==1.37.0",
    ]
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-U", "--force-reinstall"] + pkgs)

_force_pins()

## Подготовка папки `data`

Если папки нет, создадим её и положим пример файла, чтобы всё запускалось сразу.

In [ ]:
from pathlib import Path

data_dir = Path("./data")
data_dir.mkdir(parents=True, exist_ok=True)

sample_path = data_dir / "sample.txt"
if not sample_path.exists():
    sample_path.write_text(
        "LangChain помогает строить цепочки (chains), агенты и RAG‑системы.\n"
        "RAG улучшает ответы, добавляя знания из документов.\n"
        "Эмбеддинги позволяют искать похожие фрагменты текста.",
        encoding="utf-8",
    )

print("Файлы в data:", [p.name for p in data_dir.glob("*.txt")])

## Подготовка данных

Создайте папку `./data` и положите туда текстовые файлы (`.txt`).
Мы загрузим их через `DirectoryLoader` и разрежем на чанки.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = DirectoryLoader(
    "./data",
    glob="**/*.txt",
    loader_cls=TextLoader,
    show_progress=True,
)

documents = loader.load()
print(f"Загружено документов: {len(documents)}")

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)

chunks = splitter.split_documents(documents)
print(f"Всего чанков: {len(chunks)}")
print("Пример чанка:\n", chunks[0].page_content[:500])

## Разбор параметров чанкинга

- `chunk_size` — длина чанка в символах. Чем больше, тем больше контекста, но хуже точность.
- `chunk_overlap` — перекрытие между чанками. Это помогает не терять смысл на границе.

**Практические советы:**
- Начинайте с 500/50, затем подбирайте под свой домен.
- Для юридических/технических текстов иногда нужны более крупные чанки.
- Слишком маленькие чанки теряют контекст и снижают качество поиска.